In [1]:
print(1)

1


In [9]:
data = {'loss': 0.6897, 
        'grad_norm': 0.1371459499355251, 
        'learning_rate': 4.798819431378626e-07, 
        'rewards/chosen': -0.014980114996433258, 
        'rewards/rejected': -0.022156300023198128, 
        'rewards/accuracies': 0.6171875, 
        'rewards/margins': 0.007176183629781008, 
        'logps/chosen': -260.08294677734375, 
        'logps/rejected': -269.7545166015625, 
        'logits/chosen': -0.31492024660110474, 
        'logits/rejected': -0.3200848400592804,
        'epoch': 0.22}


import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))


# compute the margin
margin = data['rewards/chosen'] - data['rewards/rejected']

loss = -math.log(sigmoid(margin))
print(loss)

beta =0.01

rewards = beta * (data['logps/chosen'] - data['logps/rejected'])
print(rewards)
        
        

0.6895655252366926
0.09671569824218751


In [2]:
def analyze_dpo_metrics(data):
    """
    分析DPO训练指标之间的关系
    
    参数:
    - data: 包含DPO训练指标的字典
    
    返回:
    - analysis: 包含指标间关系分析的字典
    """
    # 提取基础指标
    loss = data['loss']
    rewards_chosen = data['rewards/chosen']
    rewards_rejected = data['rewards/rejected']
    rewards_accuracies = data['rewards/accuracies']
    rewards_margins = data['rewards/margins']
    logps_chosen = data['logps/chosen']
    logps_rejected = data['logps/rejected']
    
    # 假设beta值为0.1（常用值）
    beta = 0.1
    
    # 1. 反推参考模型的logps
    reference_logps_chosen = logps_chosen - rewards_chosen / beta
    reference_logps_rejected = logps_rejected - rewards_rejected / beta
    
    # 2. 验证rewards计算
    calculated_rewards_chosen = beta * (logps_chosen - reference_logps_chosen)
    calculated_rewards_rejected = beta * (logps_rejected - reference_logps_rejected)
    
    # 3. 验证margins计算
    calculated_margins = rewards_chosen - rewards_rejected
    margin_error = abs(calculated_margins - rewards_margins)
    
    # 4. 验证loss计算
    import math
    import numpy as np
    
    # 使用sigmoid函数计算loss
    def sigmoid(x):
        return 1 / (1 + math.exp(-x))
    
    calculated_loss = -math.log(sigmoid(rewards_margins))
    loss_error = abs(calculated_loss - loss)
    
    # 构建分析结果
    analysis = {
        "input_metrics": {
            "loss": loss,
            "rewards_chosen": rewards_chosen,
            "rewards_rejected": rewards_rejected,
            "rewards_accuracies": rewards_accuracies,
            "rewards_margins": rewards_margins,
            "logps_chosen": logps_chosen,
            "logps_rejected": logps_rejected
        },
        "derived_metrics": {
            "reference_logps_chosen": reference_logps_chosen,
            "reference_logps_rejected": reference_logps_rejected,
            "calculated_rewards_chosen": calculated_rewards_chosen,
            "calculated_rewards_rejected": calculated_rewards_rejected,
            "calculated_margins": calculated_margins,
            "margin_error": margin_error,
            "calculated_loss": calculated_loss,
            "loss_error": loss_error
        },
        "relationships": {
            "rewards_chosen_formula": "beta * (logps_chosen - reference_logps_chosen)",
            "rewards_rejected_formula": "beta * (logps_rejected - reference_logps_rejected)",
            "margins_formula": "rewards_chosen - rewards_rejected",
            "loss_formula": "-log(sigmoid(margins))"
        },
        "interpretation": {
            "accuracy_above_random": rewards_accuracies > 0.5,
            "margin_positive": rewards_margins > 0,
            "loss_decreasing": loss < 0.693,  # 0.693 ≈ ln(2) 是随机猜测的损失
            "model_learning": rewards_accuracies > 0.5 and rewards_margins > 0 and loss < 0.693
        }
    }
    
    return analysis

# 使用提供的数据
data = {
    'loss': 0.6897, 
    'grad_norm': 0.1371459499355251, 
    'learning_rate': 4.798819431378626e-07, 
    'rewards/chosen': -0.014980114996433258, 
    'rewards/rejected': -0.022156300023198128, 
    'rewards/accuracies': 0.6171875, 
    'rewards/margins': 0.007176183629781008, 
    'logps/chosen': -260.08294677734375, 
    'logps/rejected': -269.7545166015625, 
    'logits/chosen': -0.31492024660110474, 
    'logits/rejected': -0.3200848400592804, 
    'epoch': 0.22
}

# 分析指标关系
analysis = analyze_dpo_metrics(data)

# 打印详细结果
import json
print(json.dumps(analysis, indent=2))

# 简明总结
print("\n===== DPO指标关系总结 =====")
print(f"1. 奖励计算: rewards = beta * (logps - reference_logps)")
print(f"   - chosen奖励: {data['rewards/chosen']:.6f}")
print(f"   - rejected奖励: {data['rewards/rejected']:.6f}")

print(f"\n2. 边际奖励: margins = rewards_chosen - rewards_rejected")
print(f"   - 计算值: {analysis['derived_metrics']['calculated_margins']:.6f}")
print(f"   - 实际值: {data['rewards/margins']:.6f}")

print(f"\n3. 准确率: accuracy = proportion of (margins > 0)")
print(f"   - 准确率: {data['rewards/accuracies']:.4f} (约{data['rewards/accuracies']*100:.1f}%样本判断正确)")

print(f"\n4. 损失函数: loss = -log(sigmoid(margins))")
print(f"   - 计算值: {analysis['derived_metrics']['calculated_loss']:.4f}")
print(f"   - 实际值: {data['loss']:.4f}")

print(f"\n5. 训练进度: epoch = {data['epoch']:.2f}")
print(f"   - 模型学习状态: {'有效学习中' if analysis['interpretation']['model_learning'] else '尚未有效学习'}")

{
  "input_metrics": {
    "loss": 0.6897,
    "rewards_chosen": -0.014980114996433258,
    "rewards_rejected": -0.022156300023198128,
    "rewards_accuracies": 0.6171875,
    "rewards_margins": 0.007176183629781008,
    "logps_chosen": -260.08294677734375,
    "logps_rejected": -269.7545166015625
  },
  "derived_metrics": {
    "reference_logps_chosen": -259.9331456273794,
    "reference_logps_rejected": -269.5329536013305,
    "calculated_rewards_chosen": -0.014980114996433258,
    "calculated_rewards_rejected": -0.022156300023198128,
    "calculated_margins": 0.00717618502676487,
    "margin_error": 1.3969838619232178e-09,
    "calculated_loss": 0.6895655259326784,
    "loss_error": 0.0001344740673215883
  },
  "relationships": {
    "rewards_chosen_formula": "beta * (logps_chosen - reference_logps_chosen)",
    "rewards_rejected_formula": "beta * (logps_rejected - reference_logps_rejected)",
    "margins_formula": "rewards_chosen - rewards_rejected",
    "loss_formula": "-log(sigmo